# Selection of Calibration Windows for Day-Ahead Electricity Price Forecasting
## by Grzegorz Marcjasz, Tomasz Serafin and Rafał Weron 

In [116]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation

In [117]:
df_day_ahead_epex = pd.read_csv('Day_Ahead_Epex.csv', sep=';')
df_NP2018 = pd.read_csv('NP2018.csv', header=None)
df_PJM = pd.read_csv('PJM.csv', sep=';')

mmmmmmm

In [118]:
df_day_ahead_epex.head()

,Date,hour,Spot DE.AT_price,day
0,20150101,1,25.02,5
1,20150101,2,18.29,5
2,20150101,3,16.04,5
3,20150101,4,14.60,5
4,20150101,5,14.95,5


chiedere nomi colonne

In [119]:
df_NP2018.head()
columns = ['Date', 'Hour', 'Price', "placeholder", "placeholder2"]
df_NP2018.columns = columns
df_NP2018.head()

,Date,Hour,Price,placeholder,placeholder2
0,20130101,1,31.05,42497.0,2798.0
1,20130101,2,30.47,41463.0,2417.0
2,20130101,3,28.92,40812.0,2036.0
3,20130101,4,27.88,40246.0,1706.0
4,20130101,5,26.96,40088.0,1427.0


In [120]:
df_PJM.head()

,date,hour,comed price,rto load forecast,comed load foecast,day
0,20110102,0,17.141179,68590.0,11432.0,1
1,20110102,1,14.179616,65896.0,10862.0,1
2,20110102,2,12.736793,64511.0,10486.0,1
3,20110102,3,11.096262,63935.0,10318.0,1
4,20110102,4,10.690926,64321.0,10208.0,1


In [121]:
df_PJM['sunday'] = np.where(df_PJM['day'] == 7, 1, 0)
df_PJM['saturday'] = np.where(df_PJM['day'] == 6, 1, 0)
df_PJM['monday'] = np.where(df_PJM['day'] ==  1, 1, 0)


In [122]:
price = df_PJM["comed price"]
exogenos = df_PJM["rto load forecast"]
sunday = df_PJM["sunday"]
saturday = df_PJM["saturday"]
monday = df_PJM["monday"]

In [128]:
errors = []
T = 150
for h in range(24):
    p_hour = df_PJM.loc[h::24,'comed price'].values # hourly data selection
    h_errors = []
    for day in range(T,len(price)):
            cal_data = price[(day-T):day] # calibration data 
            Y = price[7:T]
            X1 = cal_data[6:T-1] 
            X2 = cal_data[5:T-2] 
            X3 = cal_data[0:T-7]
            #X4 = cal_data[0:T-1].min() # min to ask
            X5 = exogenos[0:T-7]  #C_d_h
            X6 = sunday[0:T-7] # sunday
            X7 = monday[0:T-7] # monday
            X8 = saturday[0:T-7] # saturday
            X0 = np.ones(np.size(X1)) 
            X = np.stack([X0,X1,X2, X3,X5, X6, X7, X8],axis = 1)
            betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
            X_fut = np.array([1,cal_data[T],cal_data[T-1],cal_data[T-2],cal_data[T-3],cal_data[T-3],cal_data[T-3] ,cal_data[T-3]])
            forecast = np.dot(X_fut,betas)
            real = price[day]
            err = real - forecast
            errors1 = (np.abs(err))
            errors.append(errors1)
    print("errors for model 1: " , np.mean(errors))


LinAlgError: Singular matrix

In [ ]:
Y = price[7:150]
X1 = cal_data[6:149] 
X2 = cal_data[5:150-2] 
X3 = cal_data[0:150-7]
X4 = cal_data[0:150-1].min() # min 
X5 = exogenos[7:150]  #C_d_h
X6 = sunday[7:150] # sunday
X7 = monday[7:150] # monday
X8 = saturday[7:150]

betas

### 2. Methodology

#### 2.1. Preliminaries

In [ ]:
def normalize_prices(prices):
    a = np.median(prices)
    b = median_abs_deviation(prices)
    p = 1/b * (prices - a)
    return p, a, b

def apply_vst(p): # apply the area hyperbolic sine transformation to the normalized prices
    X = np.arcsinh(p)
    return X

def inverse_transform(forecast, a, b): # apply the inverse of VST to the forecast to obtain the price prediction
    price_predictions = b * np.sinh(forecast) + a
    return price_predictions

In [ ]:
# Assuming array of prices
prices = np.array([...])

# Normalize prices
p, a, b = normalize_prices(prices)

# Apply VST
X = apply_vst(p)

# Steps for model training and forecasting go here

# Assuming forecasts is your array of forecasts
forecasts = np.array([...])

# Apply inverse transformation
price_predictions = inverse_transform(forecasts, a, b)

#### 2.2. Expert Models

##### ARX1

In [ ]:
import statsmodels.api as sm

def prepare_data_arx1(data):
    # Prepare the data for the ARX1 model
    # This includes creating the lagged variables and the dummy variables for the weekdays
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data['load_forecast'] = data['load'].shift(24)
    data = pd.get_dummies(data, columns=['weekday']) # create dummy variables for the weekdays
    data = data.dropna()  # drop missing values
    return data

def fit_arx1(data):
    # Fit the ARX1 model
    # The dependent variable is 'X'
    # The independent variables are the lagged variables, the load forecast, and the dummy variables
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'load_forecast', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog) # Ordinary Least Squares (OLS) model
    results = model.fit()
    return results # results of the model fitting

In [ ]:
errors = []
price = data["price"]
T = 150
for day in range(T,len(price)):
        cal_data = price[(day-T):day] # calibration data 
        Y = price[3:T]
        X1 = cal_data[2:T-1] 
        X2 = cal_data[0:T-2] 
        X3 = cal_data[0:T-7]
        X4 = cal_data[0:T-1] # min 
        X5 = other_variavle[0:T]  #C_d_h
        X6 = sunday[0:T] # sunday
        X7 = monday[0:T] # monday
        X8 = saturday[0:T] # tuesday
        X0 = np.ones(np.size(X1)) 
        X = np.stack([X0,X1,X2, X3, X4,X5, X6, X7, X8],axis = 1)
        betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
        X_fut = np.array([1,cal_data[T-1],cal_data[T-3]]) # modify this line to include the other variables
        forecast = np.dot(X_fut,betas)
        real = price[day]
        err = real - forecast
        errors1 = (np.abs(err))
        errors.append(errors1)
print("errors for model 1: " , np.mean(errors)

##### AR1

In [ ]:
def prepare_data_ar1(data):
    # Prepare the data for the AR1 model
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data = pd.get_dummies(data, columns=['weekday'])
    data = data.dropna()  # drop missing values
    return data

def fit_ar1(data):
    # Fit the AR1 model
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog)
    results = model.fit()
    return results

In [ ]:
# Prepare the data
data = prepare_data_ar1(data)

# Fit the AR1 model
results = fit_ar1(data)

# Print the model summary
print(results.summary())

##### ARX2

In [ ]:
def prepare_data_arx2(data):
    # Prepare the data for the ARX2 model
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data['X_max'] = data['X'].rolling(window=24).max().shift(24)
    data['X_last'] = data['X'].shift(24)
    data['load_forecast'] = data['load'].shift(24)
    data = pd.get_dummies(data, columns=['weekday'])
    data = data.dropna()  # drop missing values
    return data

def fit_arx2(data):
    # Fit the ARX2 model
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'X_max', 'X_last', 'load_forecast', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog)
    results = model.fit()
    return results

In [ ]:
# Prepare the data
data = prepare_data_arx2(data)

# Fit the ARX2 model
results = fit_arx2(data)

# Print the model summary
print(results.summary())

##### AR2

In [ ]:
def prepare_data_ar2(data):
    # Prepare the data for the AR2 model
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data['X_max'] = data['X'].rolling(window=24).max().shift(24)
    data['X_last'] = data['X'].shift(24)
    data = pd.get_dummies(data, columns=['weekday'])
    data = data.dropna()  # drop missing values
    return data

def fit_ar2(data):
    # Fit the AR2 model
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'X_max', 'X_last', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog)
    results = model.fit()
    return results

In [ ]:
# Prepare the data
data = prepare_data_ar2(data)

# Fit the AR2 model
results = fit_ar2(data)

# Print the model summary
print(results.summary())